**기본 세팅**

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

---

# 13. 과적합 피하기

이번 챕터에서는 과적합 문제를 확인하고 이를 방지하는 방법을 알아본다.

## 13.1 과적합 문제

음파 탐지기 데이터 정보를 이용해 광물과 돌을 구분하는 딥러닝 모델을 만들어 보자.

In [3]:
df = pd.read_csv("deeplearning/dataset/sonar.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

- 데이터는 샘플 208개, 60개의 피처와 타겟으로 구성되어 있다.


- 위 데이터로 딥러닝 모델을 실행 해보자.

In [5]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 시드 설정
np.random.seed(3)
tf.random.set_seed(3)

# 데이터 분리
dataset = df.values                # 타겟이 object 형태이므로 전체 행렬이 object 형태가 된다.
X = dataset[:,:60].astype(float)   # object 형태 사용시 에러
Y = dataset[:,60]

# 타겟 레이블 인코딩
encoder = LabelEncoder()
encoder.fit(Y)
Y_encoded = encoder.transform(Y)

# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# 모델 컴파일
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 실행
model.fit(X, Y_encoded, epochs=200, batch_size=5)

# 결과 출력
print("-"*100)
print(f"Accuracy: {model.evaluate(X, Y_encoded, verbose=0)[1]: .4f}")

Train on 208 samples
Epoch 1/200
208/208 [==============================] - 0s 2ms/sample - loss: 0.6825 - accuracy: 0.5673
Epoch 2/200
208/208 [==============================] - 0s 192us/sample - loss: 0.6522 - accuracy: 0.6635
Epoch 3/200
208/208 [==============================] - 0s 196us/sample - loss: 0.6283 - accuracy: 0.6971
Epoch 4/200
208/208 [==============================] - 0s 199us/sample - loss: 0.5994 - accuracy: 0.7260
Epoch 5/200
208/208 [==============================] - 0s 193us/sample - loss: 0.5708 - accuracy: 0.7356
Epoch 6/200
208/208 [==============================] - 0s 206us/sample - loss: 0.5345 - accuracy: 0.7837
Epoch 7/200
208/208 [==============================] - 0s 204us/sample - loss: 0.5111 - accuracy: 0.7740
Epoch 8/200
208/208 [==============================] - 0s 191us/sample - loss: 0.4873 - accuracy: 0.7885
Epoch 9/200
208/208 [==============================] - 0s 209us/sample - loss: 0.4623 - accuracy: 0.7644
Epoch 10/200
208/208 [==============

208/208 [==============================] - 0s 197us/sample - loss: 0.1120 - accuracy: 0.9663
Epoch 79/200
208/208 [==============================] - 0s 201us/sample - loss: 0.1028 - accuracy: 0.9712
Epoch 80/200
208/208 [==============================] - 0s 195us/sample - loss: 0.0991 - accuracy: 0.9712
Epoch 81/200
208/208 [==============================] - 0s 195us/sample - loss: 0.1189 - accuracy: 0.9471
Epoch 82/200
208/208 [==============================] - 0s 191us/sample - loss: 0.1035 - accuracy: 0.9712
Epoch 83/200
208/208 [==============================] - 0s 181us/sample - loss: 0.1008 - accuracy: 0.9760
Epoch 84/200
208/208 [==============================] - 0s 197us/sample - loss: 0.0970 - accuracy: 0.9712
Epoch 85/200
208/208 [==============================] - 0s 181us/sample - loss: 0.0918 - accuracy: 0.9760
Epoch 86/200
208/208 [==============================] - 0s 197us/sample - loss: 0.0858 - accuracy: 0.9712
Epoch 87/200
208/208 [==============================] - 0s 

208/208 [==============================] - 0s 214us/sample - loss: 0.0100 - accuracy: 1.0000
Epoch 155/200
208/208 [==============================] - 0s 223us/sample - loss: 0.0083 - accuracy: 1.0000
Epoch 156/200
208/208 [==============================] - 0s 223us/sample - loss: 0.0086 - accuracy: 1.0000
Epoch 157/200
208/208 [==============================] - 0s 236us/sample - loss: 0.0079 - accuracy: 1.0000
Epoch 158/200
208/208 [==============================] - 0s 239us/sample - loss: 0.0133 - accuracy: 1.0000
Epoch 159/200
208/208 [==============================] - 0s 226us/sample - loss: 0.0078 - accuracy: 1.0000
Epoch 160/200
208/208 [==============================] - 0s 216us/sample - loss: 0.0077 - accuracy: 1.0000
Epoch 161/200
208/208 [==============================] - 0s 220us/sample - loss: 0.0129 - accuracy: 1.0000
Epoch 162/200
208/208 [==============================] - 0s 195us/sample - loss: 0.0085 - accuracy: 1.0000
Epoch 163/200
208/208 [============================

- 예측 정확도가 100%로 이 데이터셋에 과적합 되있다.


- 교재는 `Sequential()`과 `Dense()`를 각각 `keras.models`, `keras.layers.core`로 불러온다.


- 이 경우 피처 데이터 타입을 object를 사용해도 문제없으나 `tensorflow`로 로드하면 에러가 발생한다.


- 다만 교재와 같이 실행하면 시드를 설정하여도 매번 결과가 바뀐다.


- 교재와 값이 다른 것을 넘어서 실행마다 결과가 바뀌어 패키지를 수정하였다.


- 구글링하면 global seed, operation seed를 모두 설정해야 해결 가능하다고 하는데 또 개인마다 좀 다르다고 한다.

## 13.2 Train / Test

과적합을 피하기 위해 원래 데이터를 train과 test로 나누어 모델을 실행해보자.

In [6]:
from sklearn.model_selection import train_test_split

# 시드 설정
np.random.seed(0)
tf.random.set_seed(3)

# 데이터 분리
dataset = df.values                # 타겟이 object 형태이므로 전체 행렬이 object 형태가 된다.
X = dataset[:,:60].astype(float)   # object 형태 사용시 에러
Y = dataset[:,60]

# 타겟 레이블 인코딩
encoder = LabelEncoder()
encoder.fit(Y)
Y_encoded = encoder.transform(Y)

# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.3, random_state=3)

# 모델 설정
model = Sequential()
model.add(Dense(24, input_dim=60, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# 모델 컴파일
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 실행
model.fit(X_train, Y_train, epochs=130, batch_size=5)

# 결과 출력
print("-"*100)
print(f"Accuracy: {model.evaluate(X_test, Y_test, verbose=0)[1]: .4f}")

Train on 145 samples
Epoch 1/130
145/145 [==============================] - 0s 2ms/sample - loss: 0.6841 - accuracy: 0.5241
Epoch 2/130
145/145 [==============================] - 0s 171us/sample - loss: 0.6644 - accuracy: 0.5862
Epoch 3/130
145/145 [==============================] - 0s 157us/sample - loss: 0.6478 - accuracy: 0.6345
Epoch 4/130
145/145 [==============================] - 0s 163us/sample - loss: 0.6298 - accuracy: 0.6759
Epoch 5/130
145/145 [==============================] - 0s 195us/sample - loss: 0.6117 - accuracy: 0.7241
Epoch 6/130
145/145 [==============================] - 0s 337us/sample - loss: 0.5941 - accuracy: 0.7241
Epoch 7/130
145/145 [==============================] - 0s 233us/sample - loss: 0.5756 - accuracy: 0.7241
Epoch 8/130
145/145 [==============================] - 0s 224us/sample - loss: 0.5612 - accuracy: 0.7379
Epoch 9/130
145/145 [==============================] - 0s 205us/sample - loss: 0.5394 - accuracy: 0.7172
Epoch 10/130
145/145 [==============

145/145 [==============================] - 0s 229us/sample - loss: 0.1388 - accuracy: 0.9586
Epoch 79/130
145/145 [==============================] - 0s 230us/sample - loss: 0.1253 - accuracy: 0.9724
Epoch 80/130
145/145 [==============================] - 0s 210us/sample - loss: 0.1337 - accuracy: 0.9793
Epoch 81/130
145/145 [==============================] - 0s 211us/sample - loss: 0.1251 - accuracy: 0.9724
Epoch 82/130
145/145 [==============================] - 0s 205us/sample - loss: 0.1338 - accuracy: 0.9655
Epoch 83/130
145/145 [==============================] - 0s 217us/sample - loss: 0.1162 - accuracy: 0.9793
Epoch 84/130
145/145 [==============================] - 0s 214us/sample - loss: 0.1151 - accuracy: 0.9655
Epoch 85/130
145/145 [==============================] - 0s 208us/sample - loss: 0.1094 - accuracy: 0.9793
Epoch 86/130
145/145 [==============================] - 0s 211us/sample - loss: 0.1028 - accuracy: 0.9793
Epoch 87/130
145/145 [==============================] - 0s 

- test에 대해서 정확도가 약 79.37%로 앞서 전체로 학습하고 평가 했을 때 보다 낮게 나타난다.


- test에 대한 정확도도 높은 편이라 과적합을 피해 잘 적합한 것 같다.

## 13.3 모델 저장 / 재사용

학습이 끝난 후 테스트 한 결과가 만족스럽다면 모델을 저장해서 새로운 데이터에 사용 가능하다.

In [7]:
from keras.models import load_model

# 모델 저장
model.save("my_model.h5")

# 모델 삭제
del model

# 모델 로드
model = tf.keras.models.load_model("my_model.h5")

# 결과 출력
print("-"*100)
print(f"Accuracy: {model.evaluate(X_test, Y_test, verbose=0)[1]: .4f}")

Using TensorFlow backend.


----------------------------------------------------------------------------------------------------
Accuracy:  0.7937


- 앞서 패키지에 대해 언급하였듯이 나는 `Sequential()`과 `Dense()`를 `tensorflow`에서 로드하였다.


- 만약 `kears.models`의 `load_model()`을 사용하면 Unknown initializer: GlorotUniform 라는 에러가 발생한다.


- 이유는 `tensorflow`의 keras와 `keras`는 엄연히 다르기 때문이다.


- 나는 `tensorflow`의 keras를 사용했으므로 `load_model()`역시 `tensorflow`를 사용해야 에러가 발생하지 않는다.


- 또한 `keras`의 `load_model()` 사용시 str' object has no attribute 'decode' keras 라는 에러가 발생한다면,


- `pip install h5py==2.10.0`으로 `h5py` 패키지를 다운그레이드 해준다(3이상 버전에서는 에러).

## 13.4 교차 검증

이번엔 교차 검증을 위해 `StratifiedKFold()`를 사용할 것이다.

Stratifed K-Fold는 전체 데이터의 분포도를 반영해서 train과 test로 데이터를 나눈다.

참고: [머신러닝 완벽가이드 - 사이킷런 소개](https://romg2.github.io/mlguide/01_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C-02.-%EC%82%AC%EC%9D%B4%ED%82%B7%EB%9F%B0-%EC%86%8C%EA%B0%9C/)

In [8]:
from sklearn.model_selection import StratifiedKFold

# 시드 설정
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

# 데이터 분리
dataset = df.values                # 타겟이 object 형태이므로 전체 행렬이 object 형태가 된다.
X = dataset[:,:60].astype(float)   # object 형태 사용시 에러
Y = dataset[:,60]

# 타겟 레이블 인코딩
encoder = LabelEncoder()
encoder.fit(Y)
Y_encoded = encoder.transform(Y)

# 교차 검증
accuracy = []
n_fold = 10
skf = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = seed)

for train_idx, test_idx in skf.split(X, Y_encoded):
    # 모델 설정
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation="relu"))
    model.add(Dense(10, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    # 모델 컴파일
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # 모델 실행
    model.fit(X[train_idx], Y_encoded[train_idx], epochs=100, batch_size=5)
    
    # 결과 저장
    K_accuracy = model.evaluate(X[test_idx], Y_encoded[test_idx], verbose=0)[1]
    accuracy.append(K_accuracy)

Train on 187 samples
Epoch 1/100
187/187 [==============================] - 0s 2ms/sample - loss: 0.6767 - accuracy: 0.5829
Epoch 2/100
187/187 [==============================] - 0s 179us/sample - loss: 0.6477 - accuracy: 0.6364
Epoch 3/100
187/187 [==============================] - 0s 180us/sample - loss: 0.6203 - accuracy: 0.7433
Epoch 4/100
187/187 [==============================] - 0s 185us/sample - loss: 0.5934 - accuracy: 0.7219
Epoch 5/100
187/187 [==============================] - 0s 167us/sample - loss: 0.5692 - accuracy: 0.7754
Epoch 6/100
187/187 [==============================] - 0s 172us/sample - loss: 0.5620 - accuracy: 0.7380
Epoch 7/100
187/187 [==============================] - 0s 173us/sample - loss: 0.5315 - accuracy: 0.7433
Epoch 8/100
187/187 [==============================] - 0s 184us/sample - loss: 0.4984 - accuracy: 0.8021
Epoch 9/100
187/187 [==============================] - 0s 223us/sample - loss: 0.4846 - accuracy: 0.7861
Epoch 10/100
187/187 [==============

187/187 [==============================] - 0s 180us/sample - loss: 0.1184 - accuracy: 0.9786
Epoch 79/100
187/187 [==============================] - 0s 180us/sample - loss: 0.1366 - accuracy: 0.9519
Epoch 80/100
187/187 [==============================] - 0s 175us/sample - loss: 0.1201 - accuracy: 0.9733
Epoch 81/100
187/187 [==============================] - 0s 176us/sample - loss: 0.1222 - accuracy: 0.9626
Epoch 82/100
187/187 [==============================] - 0s 187us/sample - loss: 0.1145 - accuracy: 0.9733
Epoch 83/100
187/187 [==============================] - 0s 187us/sample - loss: 0.1099 - accuracy: 0.9786
Epoch 84/100
187/187 [==============================] - 0s 173us/sample - loss: 0.1083 - accuracy: 0.9840
Epoch 85/100
187/187 [==============================] - 0s 179us/sample - loss: 0.1075 - accuracy: 0.9840
Epoch 86/100
187/187 [==============================] - 0s 180us/sample - loss: 0.1068 - accuracy: 0.9840
Epoch 87/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 244us/sample - loss: 0.3109 - accuracy: 0.8663
Epoch 56/100
187/187 [==============================] - 0s 246us/sample - loss: 0.3193 - accuracy: 0.8770
Epoch 57/100
187/187 [==============================] - 0s 231us/sample - loss: 0.3004 - accuracy: 0.8610
Epoch 58/100
187/187 [==============================] - 0s 233us/sample - loss: 0.2982 - accuracy: 0.8770
Epoch 59/100
187/187 [==============================] - 0s 233us/sample - loss: 0.2882 - accuracy: 0.8717
Epoch 60/100
187/187 [==============================] - 0s 282us/sample - loss: 0.2917 - accuracy: 0.8824
Epoch 61/100
187/187 [==============================] - 0s 285us/sample - loss: 0.2947 - accuracy: 0.8610
Epoch 62/100
187/187 [==============================] - 0s 240us/sample - loss: 0.2762 - accuracy: 0.8877
Epoch 63/100
187/187 [==============================] - 0s 234us/sample - loss: 0.2838 - accuracy: 0.8877
Epoch 64/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 194us/sample - loss: 0.3559 - accuracy: 0.8717
Epoch 33/100
187/187 [==============================] - 0s 195us/sample - loss: 0.3485 - accuracy: 0.8930
Epoch 34/100
187/187 [==============================] - 0s 188us/sample - loss: 0.3437 - accuracy: 0.8877
Epoch 35/100
187/187 [==============================] - 0s 174us/sample - loss: 0.3393 - accuracy: 0.8877
Epoch 36/100
187/187 [==============================] - 0s 191us/sample - loss: 0.3459 - accuracy: 0.8770
Epoch 37/100
187/187 [==============================] - 0s 184us/sample - loss: 0.3376 - accuracy: 0.8663
Epoch 38/100
187/187 [==============================] - 0s 185us/sample - loss: 0.3317 - accuracy: 0.8770
Epoch 39/100
187/187 [==============================] - 0s 178us/sample - loss: 0.3204 - accuracy: 0.8930
Epoch 40/100
187/187 [==============================] - 0s 181us/sample - loss: 0.3211 - accuracy: 0.8717
Epoch 41/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 184us/sample - loss: 0.5260 - accuracy: 0.7594
Epoch 10/100
187/187 [==============================] - 0s 197us/sample - loss: 0.4935 - accuracy: 0.8396
Epoch 11/100
187/187 [==============================] - 0s 183us/sample - loss: 0.4748 - accuracy: 0.8021
Epoch 12/100
187/187 [==============================] - 0s 181us/sample - loss: 0.4664 - accuracy: 0.7968
Epoch 13/100
187/187 [==============================] - 0s 201us/sample - loss: 0.4465 - accuracy: 0.8021
Epoch 14/100
187/187 [==============================] - 0s 178us/sample - loss: 0.4338 - accuracy: 0.8396
Epoch 15/100
187/187 [==============================] - 0s 178us/sample - loss: 0.4110 - accuracy: 0.8503
Epoch 16/100
187/187 [==============================] - 0s 178us/sample - loss: 0.4034 - accuracy: 0.8289
Epoch 17/100
187/187 [==============================] - 0s 186us/sample - loss: 0.4044 - accuracy: 0.8182
Epoch 18/100
187/187 [==============================] - 0s 

...

187/187 [==============================] - 0s 179us/sample - loss: 0.4130 - accuracy: 0.8289
Epoch 18/100
187/187 [==============================] - 0s 182us/sample - loss: 0.3826 - accuracy: 0.8556
Epoch 19/100
187/187 [==============================] - 0s 200us/sample - loss: 0.3814 - accuracy: 0.8610
Epoch 20/100
187/187 [==============================] - 0s 182us/sample - loss: 0.3734 - accuracy: 0.8342
Epoch 21/100
187/187 [==============================] - 0s 196us/sample - loss: 0.3604 - accuracy: 0.8342
Epoch 22/100
187/187 [==============================] - 0s 180us/sample - loss: 0.3552 - accuracy: 0.8449
Epoch 23/100
187/187 [==============================] - 0s 184us/sample - loss: 0.3319 - accuracy: 0.8396
Epoch 24/100
187/187 [==============================] - 0s 180us/sample - loss: 0.3589 - accuracy: 0.8342
Epoch 25/100
187/187 [==============================] - 0s 180us/sample - loss: 0.3347 - accuracy: 0.8449
Epoch 26/100
187/187 [==============================] - 0s 

187/187 [==============================] - 0s 188us/sample - loss: 0.0561 - accuracy: 0.9893
Epoch 95/100
187/187 [==============================] - 0s 259us/sample - loss: 0.0499 - accuracy: 0.9947
Epoch 96/100
187/187 [==============================] - 0s 196us/sample - loss: 0.0534 - accuracy: 0.9893
Epoch 97/100
187/187 [==============================] - 0s 195us/sample - loss: 0.0499 - accuracy: 1.0000
Epoch 98/100
187/187 [==============================] - 0s 197us/sample - loss: 0.0549 - accuracy: 0.9947
Epoch 99/100
187/187 [==============================] - 0s 195us/sample - loss: 0.0493 - accuracy: 0.9947
Epoch 100/100
187/187 [==============================] - 0s 437us/sample - loss: 0.0433 - accuracy: 1.0000
Train on 187 samples
Epoch 1/100
187/187 [==============================] - 0s 3ms/sample - loss: 0.6778 - accuracy: 0.5187
Epoch 2/100
187/187 [==============================] - 0s 266us/sample - loss: 0.6399 - accuracy: 0.6684
Epoch 3/100
187/187 [====================

187/187 [==============================] - 0s 464us/sample - loss: 0.1280 - accuracy: 0.9572
Epoch 72/100
187/187 [==============================] - 0s 424us/sample - loss: 0.1326 - accuracy: 0.9572
Epoch 73/100
187/187 [==============================] - 0s 275us/sample - loss: 0.1241 - accuracy: 0.9733
Epoch 74/100
187/187 [==============================] - 0s 260us/sample - loss: 0.1118 - accuracy: 0.9786
Epoch 75/100
187/187 [==============================] - 0s 317us/sample - loss: 0.1114 - accuracy: 0.9679
Epoch 76/100
187/187 [==============================] - 0s 237us/sample - loss: 0.1066 - accuracy: 0.9733
Epoch 77/100
187/187 [==============================] - 0s 297us/sample - loss: 0.1050 - accuracy: 0.9786
Epoch 78/100
187/187 [==============================] - 0s 238us/sample - loss: 0.1136 - accuracy: 0.9679
Epoch 79/100
187/187 [==============================] - 0s 215us/sample - loss: 0.1091 - accuracy: 0.9519
Epoch 80/100
187/187 [==============================] - 0s 

188/188 [==============================] - 0s 237us/sample - loss: 0.2993 - accuracy: 0.8670
Epoch 49/100
188/188 [==============================] - 0s 257us/sample - loss: 0.2795 - accuracy: 0.8989
Epoch 50/100
188/188 [==============================] - 0s 240us/sample - loss: 0.2835 - accuracy: 0.8883
Epoch 51/100
188/188 [==============================] - 0s 223us/sample - loss: 0.2838 - accuracy: 0.8989
Epoch 52/100
188/188 [==============================] - 0s 228us/sample - loss: 0.2705 - accuracy: 0.8883
Epoch 53/100
188/188 [==============================] - 0s 259us/sample - loss: 0.2857 - accuracy: 0.8936
Epoch 54/100
188/188 [==============================] - 0s 207us/sample - loss: 0.2806 - accuracy: 0.8989
Epoch 55/100
188/188 [==============================] - 0s 213us/sample - loss: 0.2672 - accuracy: 0.8989
Epoch 56/100
188/188 [==============================] - 0s 229us/sample - loss: 0.2551 - accuracy: 0.9043
Epoch 57/100
188/188 [==============================] - 0s 

188/188 [==============================] - 0s 237us/sample - loss: 0.3881 - accuracy: 0.8191
Epoch 26/100
188/188 [==============================] - 0s 231us/sample - loss: 0.3756 - accuracy: 0.8511
Epoch 27/100
188/188 [==============================] - 0s 224us/sample - loss: 0.3782 - accuracy: 0.8245
Epoch 28/100
188/188 [==============================] - 0s 388us/sample - loss: 0.3674 - accuracy: 0.8404
Epoch 29/100
188/188 [==============================] - 0s 243us/sample - loss: 0.3560 - accuracy: 0.8457
Epoch 30/100
188/188 [==============================] - 0s 241us/sample - loss: 0.3507 - accuracy: 0.8457
Epoch 31/100
188/188 [==============================] - 0s 230us/sample - loss: 0.3486 - accuracy: 0.8670
Epoch 32/100
188/188 [==============================] - 0s 228us/sample - loss: 0.3429 - accuracy: 0.8564
Epoch 33/100
188/188 [==============================] - 0s 288us/sample - loss: 0.3365 - accuracy: 0.8670
Epoch 34/100
188/188 [==============================] - 0s 

In [9]:
# 결과 출력
print(f"fold별 교차 검증 결과: {np.round(accuracy, 4)}")
print(f"{n_fold} 교차 검증 결과 평균: {np.mean(accuracy):.4f}")

fold별 교차 검증 결과: [0.7143 0.8095 0.8095 0.9048 0.7619 0.8095 0.8095 0.7619 0.9    0.85  ]
10 교차 검증 결과 평균: 0.8131


- 10 fold 교차 검증 결과 평균 정확도가 약 81.31%로 비교적 높게 나타난 듯 하다.